In [25]:
uid_score_bid = ['A,1,a','A,1,b','A,1,d','B,1,b','B,1,c','B,1,e','C,1,c','C,1,d','D,1,b','D,1,c','D,1,d','E,1,a','E,1,d']

In [26]:
#dict.setdefault : dict.setdefault(key, default=None)
#if key exist return the correspond value
#otherwise return default value
train = dict()
for rec in uid_score_bid:
    user,score,item = rec.split(",")
    train.setdefault(user,{})  
    train[user][item] = int(float(score))

In [27]:
train

{'A': {'a': 1, 'b': 1, 'd': 1},
 'B': {'b': 1, 'c': 1, 'e': 1},
 'C': {'c': 1, 'd': 1},
 'D': {'b': 1, 'c': 1, 'd': 1},
 'E': {'a': 1, 'd': 1}}

In [28]:
c = dict() #item matrix for cooccurance, if item a,b all in user mike's record
n = dict() #how many users buought this item
for user, record in train.items():
    for item in record.keys():
        n.setdefault(item, 0)
        n[item] += 1#find a record add 1
        c.setdefault(item,{})
        for other_item in record.keys():
            if other_item == item:
                continue
            c[item].setdefault(other_item, 0)
            c[item][other_item] += 1#item i j have both exist in this record

        

In [29]:
c

{'a': {'b': 1, 'd': 2},
 'b': {'a': 1, 'c': 2, 'd': 2, 'e': 1},
 'c': {'b': 2, 'd': 2, 'e': 1},
 'd': {'a': 2, 'b': 2, 'c': 2},
 'e': {'b': 1, 'c': 1}}

In [30]:
n

{'a': 2, 'b': 3, 'c': 3, 'd': 4, 'e': 1}

In [31]:
import math
#similarity metric: item i,j's coexist times/sqrt((total i time) * (total j time))
sim = dict()
for i,co_exist in c.items():
    sim.setdefault(i,{})
    for ij in co_exist.keys():
        sim[i][ij] = co_exist[ij] / math.sqrt(n[i] * n[ij])

In [32]:
sim

{'a': {'b': 0.4082482904638631, 'd': 0.7071067811865475},
 'b': {'a': 0.4082482904638631,
  'c': 0.6666666666666666,
  'd': 0.5773502691896258,
  'e': 0.5773502691896258},
 'c': {'b': 0.6666666666666666,
  'd': 0.5773502691896258,
  'e': 0.5773502691896258},
 'd': {'a': 0.7071067811865475,
  'b': 0.5773502691896258,
  'c': 0.5773502691896258},
 'e': {'b': 0.5773502691896258, 'c': 0.5773502691896258}}

In [33]:
#recommend strategy: based on histoic record for each item the user bought, for one item i
#select k item which is similiar with this item
#calculate k items recommendation score
#the score is score i from user record * similarity i,j( j from k items)
#an example for user A ,k = 3
k = 3
rank = dict()
user_record = train['A']
for item, score in user_record.items():#in this example the score 1 means the user buy this or click this item
    sorted_sim_score = sorted(sim[item].items(), key = lambda x:x[1], reverse = True)[:k]
    for other_item, sim_score in sorted_sim_score:
        if other_item in user_record.keys():
            continue
        rank.setdefault(other_item,0)
        rank[other_item] += sim_score * score
print sorted(rank.items(), key = lambda x:x[1], reverse = True)

[('c', 1.2440169358562925), ('e', 0.5773502691896258)]


In [34]:
#if a user he also is a retailer, which means he will but a lot items in this website and don't have preference
#this will impact our model performance
# add regularization: for each item have k items have relationship
#every time calculate similarity score divide the max score
c = dict() #item matrix for cooccurance, if item a,b all in user mike's record
n = dict() #how many users buought this item
for user, record in train.items():
    for item in record.keys():
        n.setdefault(item, 0)
        n[item] += 1#find a record add 1
        c.setdefault(item,{})
        for other_item in record.keys():
            if other_item == item:
                continue
            c[item].setdefault(other_item, 0)
            c[item][other_item] += 1/math.log(1+len(record)*1.0)# add penality for who buy many things
            
sim = dict()
sim_max = dict()
for i,co_exist in c.items():
    sim.setdefault(i,{})
    
    for j in co_exist.keys():
        sim_max.setdefault(j,0.0)
        sim[i][j] = co_exist[j] / math.sqrt(n[i] * n[j])
        if sim[i][j] > sim_max[j]:
            sim_max[j] = sim[i][j]
for item, sim_score_matrix in sim.items():
    for related_item in sim_score_matrix.keys():
        sim[item][related_item] = sim[item][related_item] / sim_max[related_item]

In [35]:
sim

{'a': {'b': 0.6123724356957947, 'd': 1.0},
 'b': {'a': 0.5105093993383438, 'c': 1.0, 'd': 0.721969316263228, 'e': 1.0},
 'c': {'b': 1.0, 'd': 0.8164965809277261, 'e': 1.0},
 'd': {'a': 1.0, 'b': 0.8660254037844387, 'c': 0.9794138964885573},
 'e': {'b': 0.8660254037844387, 'c': 0.8660254037844387}}

In [37]:
{1,2,3} & {2,3}

{2, 3}

In [ ]:
#声明一个ItemBased推荐的对象      
uid_score_bid='/home/lady/tmp/liushuang/1.9Item-basedCF/data/buy_user_spu.1210_0109'
Item = ItemBasedCF(uid_score_bid)#读取数据集 

M=5
pre_lst=[]
rec_lst=[]
for k in range(M): #进行5次交叉验证
    Item.splitData(k,M,seed=1)
    Item.train=Item.readData(Item.train_file)
    Item.test=Item.readData(Item.test_file) 

    Item.ItemSimilarity() #计算物品相似度矩阵 
    recommedDic = dict()
    hit = 0
    n_pre = 0
    n_rec = 0
    print '训练集数量',len(Item.train)
    print '测试集数量',len(Item.test)

    for user in  Item.train.keys():
        recommedDic[user] = Item.Recommend(user) #对于训练user生成推荐列表
        n_pre+=len(recommedDic[user])
        rec_item=set()
        for item in recommedDic[user]:
            rec_item.add(item[0])
        #测试user真实购买的商品
        buy_item=set()
        if user not in Item.test: continue
        for item in Item.test[user].keys():
            buy_item.add(item)
        hit += len(rec_item & buy_item)
    for user in Item.test:
        n_rec += len(Item.test[user])
    pre = hit/(1.0*n_pre)
    rec = hit/(1.0*n_rec)
    pre_lst.append(pre)
    rec_lst.append(rec)
    print k,' hit:',hit,'n_pre:',n_pre,'n_rec;',n_rec
print pre_lst,'平均：',np.mean(pre_lst)
print rec_lst,'平均：',np.mean(rec_lst)


In [49]:
with open('/Users/cni/PycharmProjects/recommender_system/data/ml-100k/u.data') as f:
    lines = f.readlines()

In [40]:
import random

In [51]:
def splitdata( k, lines, m=5,seed=1):
    train = []
    test = []
    random.seed(seed)
    for line in lines:
        if random.randint(0,m) == 1:
            test.append(line)
        else:
            train.append(line)
    return train,test

In [52]:
train, test = splitdata(1,lines)

In [53]:
len(train)

83446

In [54]:
len(test)

16554

In [56]:
train[0].strip('\n').split('\t')

['196', '242', '3', '881250949']

In [85]:
from item_based import ItemBased

In [86]:
filepath = '/Users/cni/PycharmProjects/recommender_system/data/ml-100k/u.data'

In [87]:
movelens = ItemBased(filepath)

In [88]:
movelens.train_test_split()

load file 0 lines complete
load file 10000 lines complete
load file 20000 lines complete
load file 30000 lines complete
load file 40000 lines complete
load file 50000 lines complete
load file 60000 lines complete
load file 70000 lines complete
load file 80000 lines complete
load file 90000 lines complete


In [62]:
movelens.generate_sim_matrix()

In [80]:
movelens.test_data

{}

In [69]:
random.random()

0.2550690257394217

In [90]:
random.choice(movelens.train_data.keys())

'114'

In [93]:
with open('/Users/cni/PycharmProjects/recommender_system/data/ml-100k/u.item') as f:
    lines = f.readlines()

In [97]:
lines[0]

'1|Toy Story (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)|0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0\n'

In [98]:
a =set()

In [99]:
a.add(1)

In [100]:
a

{1}